In [4]:

import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
from scipy import optimize

from tyssue import (config, Sheet,
                    SheetGeometry as geom)

from tyssue.dynamics.sheet_isotropic_model import isotropic_relax
from tyssue.dynamics.sheet_vertex_model import SheetModel as model
from tyssue.solvers.sheet_vertex_solver import Solver as solver

from tyssue.draw.plt_draw import sheet_view, quick_edge_draw
from tyssue.io import hdf5
from tyssue.io import obj



In [11]:
datasets = hdf5.load_datasets('../data/hf5/before_apoptosis.hf5')

sheet = Sheet('pola', datasets,
              config.geometry.cylindrical_sheet())

geom.update_all(sheet)


nondim_specs = config.dynamics.quasistatic_sheet_spec()
dim_model_specs = model.dimentionalize(nondim_specs)

sheet.update_specs(dim_model_specs)
isotropic_relax(sheet, nondim_specs)


In [12]:
sheet.settings


{'geometry': 'cylindrical',
 'grad_norm_factor': 23943.87036383216,
 'height_axis': 'z',
 'nrj_norm_factor': 82944.0}

In [13]:
%matplotlib inline

```python
local_mask_vert = (sheet.vert_df.x.abs() < 20) & (sheet.vert_df.z.abs() < 10) & (sheet.vert_df.y > 0)
local_patch_vert = sheet.vert_df[local_mask_vert] 
local_patch_edge = sheet.edge_df[sheet.upcast_srce(local_mask_vert) & sheet.upcast_trgt(local_mask_vert)]
local_patch_face = sheet.face_df.loc[np.unique(local_patch_edge.index.get_level_values('face'))]
local_datasets = {'vert': local_patch_vert,
                  'edge': local_patch_edge,
                  'face':local_patch_face}
local_sheet = Sheet('local', local_datasets)
```

In [14]:
sheet.vert_df.head()

,is_active,z,x,y,old_idx,basal_shift,rho,height,radial_tension
jv,,,,,,,,,
0,0,-64.822644,36.776656,-0.533739,928,30.015534,36.780529,6.764995,0.0
1,0,-63.906322,37.194778,2.748014,929,30.015534,37.296154,7.280621,0.0
2,0,-65.217397,36.228740,5.999018,930,30.015534,36.722062,6.706528,0.0
3,0,-62.954196,37.261064,-3.672709,931,30.015534,37.441631,7.426097,0.0
4,1,-61.266034,37.410892,-3.473537,932,30.015534,37.571802,7.556269,0.0


In [15]:
live_edge = sheet.upcast_face(sheet.face_df.is_alive).astype(np.bool)
dead_edge = np.logical_not(live_edge)
inactive_verts = set(sheet.edge_df[dead_edge].srce)

active_verts = set(sheet.vert_df.index).difference(inactive_verts)
active_verts = list(active_verts)
sheet.vert_df.is_active = 0
sheet.vert_df.loc[active_verts, 'is_active'] = 1
# sheet.vert_df.is_active = 1

In [ ]:
# vp_view(sheet, coords=['z', 'x', 'y'])
# obj.write_splitted_cells(sheet, 'test_pola.obj')

In [ ]:

draw_specs = load_default('draw', 'sheet')
draw_specs['vert']['visible'] = False
draw_specs['edge']['width'] = 0.2
draw_specs['edge']['width'] = sheet.upcast_srce(sheet.vert_df.y > 0).astype(np.float)*0.5

fig, ax = sheet_view(sheet, ['z', 'x'], **draw_specs)
ax.set_xlim(sheet.vert_df.z.min()-1, sheet.vert_df.z.max()+1)
ax.set_ylim(sheet.vert_df.x.min()-1, sheet.vert_df.x.max()+1)
fig.set_size_inches(12, 12)
plt.savefig('test_pola.png')



In [ ]:

settings = {
    'norm_factor': 1,
    'minimize': {
        'method': 'L-BFGS-B',
        'options': {'disp': False,
                    'ftol': 1e-6,
                    'gtol': 1e-6},
        }
    }


res = solver.find_energy_min(sheet, geom, model, **settings)
print(res['success'])

In [ ]:
# center_edges = sheet.edge_df[sheet.upcast_srce(sheet.vert_df['z']).abs() < 10].index

# sheet.edge_df.loc[center_edges, 'line_tension'] = (sheet.edge_df.loc[center_edges, 'line_tension'] * 
#                                               (5*np.sin(sheet.edge_df.loc[center_edges, 'nz'])**2))

orient = sheet.edge_df['dz'] /  sheet.edge_df['length']
sheet.edge_df['line_tension'] = (sheet.specs['edge']['line_tension'] * 
                                 2*(orient**2))

fig, ax = plt.subplots()
ax.plot(sheet.upcast_srce(sheet.vert_df['z']), sheet.edge_df['line_tension'], '.', alpha=0.2)

In [ ]:
settings = {
    'minimize': {
        'options': {'disp':False,
        'gtol':1e-4,
        'ftol':1e-4},
        }
    }
res = solver.find_energy_min(sheet,  geom, model, pos_idx=active_verts, **settings)
print(res['success'])

In [ ]:
draw_specs['vert']['visible'] = False


edge_color = sheet.edge_df['line_tension'] / sheet.edge_df['line_tension'].max()
cmap = plt.cm.get_cmap('viridis')
edge_cmap = cmap(edge_color)
draw_specs['edge']['color'] = edge_cmap
draw_specs['edge']['width'] = sheet.upcast_srce(sheet.vert_df.y > 0).astype(np.float)


fig, ax = sheet_view(sheet, ['z', 'x'], **draw_specs)
ax.set_xlim(sheet.vert_df.z.min() - 1,
            sheet.vert_df.z.max() + 1)
ax.set_ylim(sheet.vert_df.x.min() - 1,
            sheet.vert_df.x.max() + 1)


In [ ]:
local_mask_vert = (sheet.vert_df.x.abs() < 20) & (sheet.vert_df.z.abs() < 20) & (sheet.vert_df.y > 0)
local_patch_vert = sheet.vert_df[local_mask_vert] 
local_patch_edge = sheet.edge_df[sheet.upcast_srce(local_mask_vert) & sheet.upcast_trgt(local_mask_vert)]
local_patch_cell = sheet.face_df.loc[np.unique(local_patch_edge.face)]
local_datasets = {'vert': local_patch_vert,
                  'edge': local_patch_edge,
                  'face': local_patch_cell}
local_sheet = Sheet('local', local_datasets)

fig, ax = sheet_view(local_sheet, ['z', 'x'], **draw_specs)
ax.set_xlim(local_sheet.vert_df.z.min()-1, local_sheet.vert_df.z.max()+1)
ax.set_ylim(local_sheet.vert_df.x.min()-1, local_sheet.vert_df.x.max()+1)

In [ ]:
h = plt.hist(sheet.edge_df.length, bins=100)